<a href="https://colab.research.google.com/github/Sajithpemarathna/Inventory-business-case/blob/main/notebooks/inventory_case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install duckdb --upgrade

import pandas as pd
import numpy as np
import duckdb
from pathlib import Path
from datetime import datetime
pd.set_option('display.max_columns', 100)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.5/20.5 MB 44.6 MB/s eta 0:00:00


In [ ]:
RAW_URL = "https://raw.githubusercontent.com/Sajithpemarathna/Inventory-business-case/main/data/raw/Inventory%20Management%20Business%20Case%20data%20-%20Sheet1.csv"

# Read CSV directly from GitHub Raw
df_raw = pd.read_csv(RAW_URL)
df_raw.head()


,Car unique id,first online date,car sold on date,manufacturer,model,fuel_type,buy price,last available online price,External market reference price
0,1234768090,2024-10-17,2024-11-18,Volkswagen,T6 Multivan,Diesel,35031.0,43688,43895.0
1,1258374660,2024-11-26,NaN,MINI,MINI,Benzin,11504.0,16623,15321.0
2,1272940076,2024-10-01,2024-12-07,Mitsubishi,Eclipse Cross,Benzin,13899.0,18422,17831.0
3,1275633463,2024-09-20,2024-11-05,Ford,Fiesta,Benzin,8522.0,13727,13294.0
4,1248774080,2024-11-05,NaN,BMW,X1,Benzin,11564.0,16594,16781.0


In [ ]:
# Rename columns to snake_case - Consistent, code-friendly names reduce types and make SQL/Python joins & references reliable.
df = df_raw.rename(columns={
    'Car unique id': 'car_id',
    'first online date': 'first_online_date',
    'car sold on date': 'car_sold_on_date',
    'manufacturer': 'manufacturer',
    'model': 'model',
    'fuel_type': 'fuel_type',
    'buy price': 'buy_price',
    'last available online price': 'last_online_price',
    'External market reference price': 'market_reference_price'
}).copy()

# Convert date columns to datetime - Ensures we can compute durations (e.g., days_online) and filter by time correctly.
for c in ['first_online_date','car_sold_on_date']:
    df[c] = pd.to_datetime(df[c], errors='coerce')

# Convert price columns to numeric - Guarantees arithmetic works (margins, price gaps) and avoids string/format issues.
for c in ['buy_price','last_online_price','market_reference_price']:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Quick peek

df.head(5)


,car_id,first_online_date,car_sold_on_date,manufacturer,model,fuel_type,buy_price,last_online_price,market_reference_price
0,1234768090,2024-10-17,2024-11-18,Volkswagen,T6 Multivan,Diesel,35031.0,43688,43895.0
1,1258374660,2024-11-26,NaT,MINI,MINI,Benzin,11504.0,16623,15321.0
2,1272940076,2024-10-01,2024-12-07,Mitsubishi,Eclipse Cross,Benzin,13899.0,18422,17831.0
3,1275633463,2024-09-20,2024-11-05,Ford,Fiesta,Benzin,8522.0,13727,13294.0
4,1248774080,2024-11-05,NaT,BMW,X1,Benzin,11564.0,16594,16781.0


In [ ]:
# Count duplicates on the unique key - Duplicate car_ids would double-count inventory or sales; they must be 0 or removed if found.
dup_count = int(df['car_id'].duplicated().sum())

# Null overview for key fields - Missing values drive our handling strategy (e.g., unsold cars, unknown buy_price).
null_buy_price = int(df['buy_price'].isna().sum())
null_sold_date = int(df['car_sold_on_date'].isna().sum())

# Sanity check for nonpositive prices - Zero/negative prices indicate data-entry errors; we’ll flag them to exclude from margin KPIs if needed.
nonpos_buy   = int((df['buy_price']<=0).fillna(False).sum())
nonpos_last  = int((df['last_online_price']<=0).fillna(False).sum())
nonpos_mref  = int((df['market_reference_price']<=0).fillna(False).sum())

# Compact report to show in the notebook
dq = pd.DataFrame({
    'metric': [
        'rows',
        'duplicate_car_id',
        'null_buy_price',
        'null_car_sold_on_date',
        'nonpositive_buy_price',
        'nonpositive_last_online_price',
        'nonpositive_market_reference_price'
    ],
    'value': [
        len(df),
        dup_count,
        null_buy_price,
        null_sold_date,
        nonpos_buy,
        nonpos_last,
        nonpos_mref
    ]
})
dq


,metric,value
0,rows,1000
1,duplicate_car_id,1
2,null_buy_price,4
3,null_car_sold_on_date,544
4,nonpositive_buy_price,0
5,nonpositive_last_online_price,0
6,nonpositive_market_reference_price,0


In [ ]:
# Inspect duplicate records
dup_ids = df['car_id'][df['car_id'].duplicated()].unique()
dup_ids


array([1282763617])

In [ ]:
# Show all rows for those duplicate car_ids
df[df['car_id'].isin(dup_ids)].sort_values('car_id')


,car_id,first_online_date,car_sold_on_date,manufacturer,model,fuel_type,buy_price,last_online_price,market_reference_price
454,1282763617,2024-09-13,2024-09-16,Volkswagen,Polo,Benzin,10902.0,12699,13426.0
567,1282763617,2024-09-13,NaT,Volkswagen,Polo,Benzin,10902.0,12000,13426.0


In [ ]:
# Remove outdated duplicate based on sold status
# Two records share the same car_id, meaning the car appeared twice.
# The record with car_sold_on_date IS the final business truth (sale completed).
# The record without the sale date represents an older state.
# We keep the sold record to maintain accurate inventory and prevent double-counting.

# Identify the duplicated car_id found earlier
dup_id = dup_ids[0]

# Drop only the duplicate row where sold date is missing
df = df[~((df['car_id'] == dup_id) & (df['car_sold_on_date'].isna()))]

# Confirm duplicates are resolved
df['car_id'].duplicated().sum()


np.int64(0)

In [ ]:
# Enables sell-through rate analysis (how many cars converted into revenue)
df['is_sold'] = df['car_sold_on_date'].notna().astype(int)


In [ ]:
df['is_sold']

,is_sold
0,1
1,0
2,1
3,1
4,0
...,...
995,0
996,1
997,0
998,0


In [ ]:
# Missing buy price - Margin cannot be computed without cost
df['missing_buy_price'] = df['buy_price'].isna()


In [ ]:
df['missing_buy_price']

,missing_buy_price
0,False
1,False
2,False
3,False
4,False
...,...
995,False
996,False
997,False
998,False


In [ ]:
df[df['missing_buy_price']=='True']

,car_id,first_online_date,car_sold_on_date,manufacturer,model,fuel_type,buy_price,last_online_price,market_reference_price,is_sold,missing_buy_price


In [ ]:
# Days Online - Inventory age drives pricing, demand decay, and cash flow impact
today = pd.Timestamp.today().normalize()
effective_end = np.where(df['is_sold']==1, df['car_sold_on_date'], today)
df['days_online'] = (pd.to_datetime(effective_end) - df['first_online_date']).dt.days


In [ ]:
df

,car_id,first_online_date,car_sold_on_date,manufacturer,model,fuel_type,buy_price,last_online_price,market_reference_price,is_sold,missing_buy_price,days_online
0,1234768090,2024-10-17,2024-11-18,Volkswagen,T6 Multivan,Diesel,35031.0,43688,43895.0,1,False,32
1,1258374660,2024-11-26,NaT,MINI,MINI,Benzin,11504.0,16623,15321.0,0,False,339
2,1272940076,2024-10-01,2024-12-07,Mitsubishi,Eclipse Cross,Benzin,13899.0,18422,17831.0,1,False,67
3,1275633463,2024-09-20,2024-11-05,Ford,Fiesta,Benzin,8522.0,13727,13294.0,1,False,46
4,1248774080,2024-11-05,NaT,BMW,X1,Benzin,11564.0,16594,16781.0,0,False,360
...,...,...,...,...,...,...,...,...,...,...,...,...
995,1282778585,2024-11-16,NaT,Volkswagen,Polo,Benzin,8454.0,10498,10892.0,0,False,349
996,1283427265,2024-12-13,2024-12-24,Peugeot,Partner,Diesel,10313.0,14685,15999.0,1,False,11
997,1282981132,2024-11-19,NaT,BMW,X1,Benzin,17946.0,20845,21195.0,0,False,346
998,1283678592,2024-12-13,NaT,Opel,Insignia Sports Tourer,Diesel,13785.0,17910,18939.0,0,False,322


In [ ]:
# Profitability metrics - Measures real business value generated per sale
df['gross_margin_eur'] = df['last_online_price'] - df['buy_price']
df['profit_margin_pct'] = round(df['gross_margin_eur']*100 / df['buy_price'],2)


In [ ]:
df

,car_id,first_online_date,car_sold_on_date,manufacturer,model,fuel_type,buy_price,last_online_price,market_reference_price,is_sold,missing_buy_price,days_online,gross_margin_eur,profit_margin_pct
0,1234768090,2024-10-17,2024-11-18,Volkswagen,T6 Multivan,Diesel,35031.0,43688,43895.0,1,False,32,8657.0,24.71
1,1258374660,2024-11-26,NaT,MINI,MINI,Benzin,11504.0,16623,15321.0,0,False,339,5119.0,44.50
2,1272940076,2024-10-01,2024-12-07,Mitsubishi,Eclipse Cross,Benzin,13899.0,18422,17831.0,1,False,67,4523.0,32.54
3,1275633463,2024-09-20,2024-11-05,Ford,Fiesta,Benzin,8522.0,13727,13294.0,1,False,46,5205.0,61.08
4,1248774080,2024-11-05,NaT,BMW,X1,Benzin,11564.0,16594,16781.0,0,False,360,5030.0,43.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1282778585,2024-11-16,NaT,Volkswagen,Polo,Benzin,8454.0,10498,10892.0,0,False,349,2044.0,24.18
996,1283427265,2024-12-13,2024-12-24,Peugeot,Partner,Diesel,10313.0,14685,15999.0,1,False,11,4372.0,42.39
997,1282981132,2024-11-19,NaT,BMW,X1,Benzin,17946.0,20845,21195.0,0,False,346,2899.0,16.15
998,1283678592,2024-12-13,NaT,Opel,Insignia Sports Tourer,Diesel,13785.0,17910,18939.0,0,False,322,4125.0,29.92


In [ ]:
# Created age buckets

# Define buckets: 0–7, 8–14, 15–30, 31–60, 61–90, 91–180, 181–365, 365+
bins   = [-1, 7, 14, 30, 60, 90, 180, 365, float('inf')]
labels = ['0-7','8-14','15-30','31-60','61-90','91-180','181-365','365+']

df['age_bucket'] = pd.cut(df['days_online'], bins=bins, labels=labels, right=True, include_lowest=True)

# Quick check
df[['car_id','days_online','age_bucket']].head(10)


,car_id,days_online,age_bucket
0,1234768090,32,31-60
1,1258374660,339,181-365
2,1272940076,67,61-90
3,1275633463,46,31-60
4,1248774080,360,181-365
5,1281970904,78,61-90
6,1264815056,23,15-30
7,1272677082,53,31-60
8,1283265418,339,181-365
9,1278123756,62,61-90


In [ ]:
df

,car_id,first_online_date,car_sold_on_date,manufacturer,model,fuel_type,buy_price,last_online_price,market_reference_price,is_sold,missing_buy_price,days_online,gross_margin_eur,profit_margin_pct,age_bucket
0,1234768090,2024-10-17,2024-11-18,Volkswagen,T6 Multivan,Diesel,35031.0,43688,43895.0,1,False,32,8657.0,24.71,31-60
1,1258374660,2024-11-26,NaT,MINI,MINI,Benzin,11504.0,16623,15321.0,0,False,339,5119.0,44.50,181-365
2,1272940076,2024-10-01,2024-12-07,Mitsubishi,Eclipse Cross,Benzin,13899.0,18422,17831.0,1,False,67,4523.0,32.54,61-90
3,1275633463,2024-09-20,2024-11-05,Ford,Fiesta,Benzin,8522.0,13727,13294.0,1,False,46,5205.0,61.08,31-60
4,1248774080,2024-11-05,NaT,BMW,X1,Benzin,11564.0,16594,16781.0,0,False,360,5030.0,43.50,181-365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1282778585,2024-11-16,NaT,Volkswagen,Polo,Benzin,8454.0,10498,10892.0,0,False,349,2044.0,24.18,181-365
996,1283427265,2024-12-13,2024-12-24,Peugeot,Partner,Diesel,10313.0,14685,15999.0,1,False,11,4372.0,42.39,8-14
997,1282981132,2024-11-19,NaT,BMW,X1,Benzin,17946.0,20845,21195.0,0,False,346,2899.0,16.15,181-365
998,1283678592,2024-12-13,NaT,Opel,Insignia Sports Tourer,Diesel,13785.0,17910,18939.0,0,False,322,4125.0,29.92,181-365


In [ ]:
 # Save enhanced feature-engineered dataset
output_path = '/content/sample_data/car_level_Layer_1_data_cleaning_&_feature_engineering.csv'
df.to_csv(output_path, index=False)